In [9]:
import pandas as pd
import numpy as np
from config import *
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [10]:
df = pd.read_parquet(DATA_DIR / "anti_saccade_processed.pq")

df.head()

,experiment,participant_id,trial_id,time,event,colour,stimulus_x,stimulus_y,eye,start_time,...,avg_pupil_size,start_x,start_y,end_x,end_y,amplitude,peak_velocity,side,time_elapsed,delay
1012,ANTI_SACCADE,106,0.0,2281646.0,TRIALID,None,NaN,NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1013,ANTI_SACCADE,106,0.0,2281671.0,START,None,NaN,NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1014,ANTI_SACCADE,106,0.0,2281678.0,SFIX,None,NaN,NaN,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1015,ANTI_SACCADE,106,0.0,2281678.0,SFIX,None,NaN,NaN,R,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1016,ANTI_SACCADE,106,0.0,2281706.0,FIXPOINT,255 255 255,960.0,540.0,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None


In [12]:
df

,experiment,participant_id,trial_id,time,event,colour,stimulus_x,stimulus_y,eye,start_time,...,avg_pupil_size,start_x,start_y,end_x,end_y,amplitude,peak_velocity,side,time_elapsed,delay
1012,ANTI_SACCADE,106,0.0,2281646.0,TRIALID,None,NaN,NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1013,ANTI_SACCADE,106,0.0,2281671.0,START,None,NaN,NaN,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1014,ANTI_SACCADE,106,0.0,2281678.0,SFIX,None,NaN,NaN,L,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1015,ANTI_SACCADE,106,0.0,2281678.0,SFIX,None,NaN,NaN,R,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
1016,ANTI_SACCADE,106,0.0,2281706.0,FIXPOINT,255 255 255,960.0,540.0,None,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124574,ANTI_SACCADE,404,15.0,NaN,ESACC,None,NaN,NaN,R,3775557.0,...,NaN,1984.9,NaN,2159.3,NaN,5.25,453.0,None,None,None
124575,ANTI_SACCADE,404,15.0,NaN,ESACC,None,NaN,NaN,L,3775564.0,...,NaN,1877.9,28.1,1945.0,NaN,1.30,179.0,None,None,None
124576,ANTI_SACCADE,404,15.0,NaN,EFIX,None,NaN,NaN,R,3775597.0,...,871.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None
124577,ANTI_SACCADE,404,15.0,NaN,EFIX,None,NaN,NaN,L,3775599.0,...,1080.0,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None


In [13]:
df_grouped = df.groupby(["experiment", "participant_id"])
features = df_grouped[["peak_velocity", "amplitude", "duration", "avg_pupil_size"]].agg([np.min, np.max, np.mean, np.median, np.std])

/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_13340/3871966300.py:2: FutureWarning: The provided callable <function min at 0x1107d3240> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  features = df_grouped[["peak_velocity", "amplitude", "duration", "avg_pupil_size"]].agg([np.min, np.max, np.mean, np.median, np.std])
/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_13340/3871966300.py:2: FutureWarning: The provided callable <function max at 0x1107d3100> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  features = df_grouped[["peak_velocity", "amplitude", "duration", "avg_pupil_size"]].agg([np.min, np.max, np.mean, np.median, np.std])
/var/folders/v0/l_dtghc15651j6_9p3clc_r00000gt/T/ipykernel_13340/3871966300.py:2: FutureWa

# Load outcome

In [14]:
demographics = pd.read_excel(DATA_DIR / "demographic_info.xlsx")[["ID", "Group"]]

demographics["y"] = (demographics["Group"] == "PATIENT").astype(int)

# Model training

In [ ]:
data = features.join(demographics, how = 'left', on='participant_id').drop("participant_id")
X_data = data.drop("y")
y_data = data["y"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=.2)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", XGBClassifier(n_estimators=10, max_depth=6, learning_rate=1, objective='binary:logistic'))
])

pipe.fit(X_train, y_train).score(X_test, y_test)

results=pd.DataFrame()
results['columns']=X_train.columns
results['importances'] = pipe["clf"].feature_importances_
results.sort_values(by='importances',ascending=False,inplace=True)

results